In [1]:
import os
from pathlib import Path
from typing import Optional
import numpy as np
from typing import List, Optional, Tuple
import matplotlib.pyplot as plt
import random
import pandas as pd
import cv2

import lightning as L
import torch
from torch.utils.data import DataLoader
from torchmetrics import JaccardIndex

from minerva.models.nets.image.sam import Sam
from minerva.data.datasets.supervised_dataset import SimpleDataset
from minerva.data.readers.png_reader import PNGReader
from minerva.data.readers.tiff_reader import TiffReader
from minerva.transforms.transform import _Transform
from minerva.data.readers.reader import _Reader

from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("PyTorch Version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA Device")

PyTorch Version: 2.5.1+cu124
CUDA Version: 12.4
CUDA Available: True
CUDA Device Count: 1
CUDA Device Name: NVIDIA GeForce RTX 4090


## variables

In [3]:
# f3
train_path = "/workspaces/Minerva-Discovery/shared_data/seismic/f3_segmentation/images"
annotation_path = "/workspaces/Minerva-Discovery/shared_data/seismic/f3_segmentation/annotations"

# parihaka
# train_path = "/workspaces/Minerva-Discovery/shared_data/seam_ai_datasets/seam_ai/images"
# annotation_path = "/workspaces/Minerva-Discovery/shared_data/seam_ai_datasets/seam_ai/annotations"

# checkpoints SAM
checkpoint_path = "/workspaces/Minerva-Discovery/shared_data/weights_sam/checkpoints_sam/sam_vit_b_01ec64.pth" # vit_b
# checkpoint_path = "/workspaces/Minerva-Discovery/shared_data/weights_sam/checkpoints_sam/sam_vit_h_4b8939.pth" # vit_h

model_name = "SAM-ViT_B_with_prompts"
vit_model = 'vit-b'
height, width = 255, 701 # f3
# height, width = 1006, 590 # parihaka
multimask_output=False # estamos fazendo inferencia para o acerto de 1 unica mascara (no SAM, a melhor mascara possivel dado as num_classes possiveis)
num_classes = 3
num_points = 3
num_epochs = 15
batch_size = 1 # deixa sempre como 1 de preferencia

In [4]:
def plot_all(image, label, pred, diff, score, point_coords, point_labels):
    """
    Plota as imagens lado a lado: imagem original, label, predição, diff.
    Pontos acumulados são exibidos sobre as imagens.
    """
    num_subplots = 4  # Número de subplots: imagem original, label, pred, diff
    plt.clf()
    fig, axes = plt.subplots(1, num_subplots, figsize=(5 * num_subplots, 5))

    # Plot 1: Imagem original
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    # Plot 2: Label
    axes[1].imshow(label, cmap='gray')
    axes[1].set_title("Label")
    axes[1].axis('off')

    # Plot 3: Predição acumulada
    axes[2].imshow(pred, cmap='gray')
    axes[2].set_title(f"Pred - Score: {score}")
    axes[2].axis('off')

    # Plot 4: Diferença entre label e pred
    axes[3].imshow(diff, cmap='gray')
    axes[3].set_title("Difference (Label - Pred)")
    axes[3].axis('off')

    # Adiciona os pontos em todas as imagens
    for ax in axes:
        for (x, y), label in zip(point_coords, point_labels):
            color = 'green' if label == 1 else 'red'
            ax.scatter(x, y, color=color, s=50, edgecolors='white')

    plt.tight_layout()
    plt.show()

## transform

In [5]:
class Padding(_Transform):
    def __init__(self, target_h_size: int, target_w_size: int):
        self.target_h_size = target_h_size
        self.target_w_size = target_w_size

    def __call__(self, x: np.ndarray) -> np.ndarray:
        h, w = x.shape[:2]
        pad_h = max(0, self.target_h_size - h)
        pad_w = max(0, self.target_w_size - w)
        if len(x.shape) == 2:
            padded = np.pad(x, ((0, pad_h), (0, pad_w)), mode="reflect")
            padded = np.expand_dims(padded, axis=2)
            padded = torch.from_numpy(padded).float()
        else:
            padded = np.pad(x, ((0, pad_h), (0, pad_w), (0, 0)), mode="reflect")
            padded = torch.from_numpy(padded).float()

        padded = np.transpose(padded, (2, 0, 1))
        return padded

## Dataset (ignore prompts and return full label and your image)

In [6]:
# versao que usa só o dict da imagem, label e o size da imagem
class DatasetForSAM(SimpleDataset):
    def __init__(
            self, 
            readers: List[_Reader], 
            transforms: Optional[_Transform] = None,
    ):
        """
        Custom Dataset to use properties that needed in images when send some image to SAM model.

        Parameters
        ----------
        readers: List[_Reader]
            List of data readers. It must contain exactly 2 readers.
            The first reader for the input data and the second reader for the
            target data.
        transforms: Optional[_Transform]
            Optional data transformation pipeline.
        transform_coords_input: Optional[dict] 
            List with transforms to apply.
                point_coords (np.ndarray or None): A Nx2 array of point prompts to the model. Each point is in (X,Y) in pixels.
                point_labels (np.ndarray or None): A length N array of labels for the point prompts. 1 indicates a foreground point and 0 indicates a background point.
    """
        super().__init__(readers, transforms)

        assert (
            len(self.readers) == 2
        ), "DatasetForSAM requires exactly 2 readers (image your label)"

        # assert (
        #     len(self.readers) == len(self.transforms)
        #     and len(self.transforms) == len(self.transform_coords_input)
        #     and len(self.readers) == len(self.transform_coords_input)
        # ), "DatasetForSAM requires exactly iquals lens (readers, transforms and transform_coords_input)"
    
    def __getitem__(self, index: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Load data and return data with SAM format (dict), where dict has:
        'image' (required): The image as a torch tensor in 3xHxW format.
        'label' (required): The label of the image.
        'original_size' (required): The original size of the image before transformation.
        'point_coords' (optional): (torch.Tensor) Batched point prompts for this image, with shape BxNx2. Already transformed to the input frame of the model.
        'point_labels' (optional): (torch.Tensor) Batched labels for point prompts, with shape BxN. (0 is background, 1 is object and -1 is pad)
        'boxes' (optional): (torch.Tensor) Batched box inputs, with shape Bx4.  Already transformed to the input frame of the model.
        'mask_inputs' (optional): (torch.Tensor) Batched mask inputs to the model, in the form Bx1xHxW.
        """

        data_readers = []
        for reader, transform in zip(self.readers, self.transforms):
            sample = reader[index]
            if transform is not None:
                sample = transform(sample)
            data_readers.append(sample)
        
        data = {
            'image': data_readers[0],
            'label': data_readers[1],
            'original_size': (int(data_readers[0].shape[1]), int(data_readers[0].shape[2]))
        }
    
        return data

## Datamodule

In [7]:
class DataModule(L.LightningDataModule):
    def __init__(
        self,
        train_path: str,
        annotations_path: str,
        transforms: _Transform = None,
        batch_size: int = 1,
        data_ratio: float = 1.0,
        num_workers: int = None,
    ):
        super().__init__()
        self.train_path = Path(train_path)
        self.annotations_path = Path(annotations_path)
        self.transforms = transforms
        self.batch_size = batch_size
        self.data_ratio = data_ratio
        self.num_workers = (
            num_workers if num_workers is not None else os.cpu_count()
        )
        print("num_workers: ", self.num_workers)

        self.datasets = {}

    def setup(self, stage=None):
        if stage == "fit":
            train_img_reader = TiffReader(self.train_path / "train")
            train_label_reader = PNGReader(self.annotations_path / "train")

            # applying ratio
            num_train_samples = int(len(train_img_reader) * self.data_ratio)
            if num_train_samples < len(train_img_reader):
                indices = random.sample(range(len(train_img_reader)), num_train_samples)
                train_img_reader = [train_img_reader[i] for i in indices]
                train_label_reader = [train_label_reader[i] for i in indices]
                
            train_dataset = DatasetForSAM(
                readers=[train_img_reader, train_label_reader],
                transforms=self.transforms
            )

            val_img_reader = TiffReader(self.train_path / "val")
            val_label_reader = PNGReader(self.annotations_path / "val")
            val_dataset = DatasetForSAM(
                readers=[val_img_reader, val_label_reader],
                transforms=self.transforms
            )

            self.datasets["train"] = train_dataset
            self.datasets["val"] = val_dataset

        elif stage == "test" or stage == "predict":
            test_img_reader = TiffReader(self.train_path / "test")
            test_label_reader = PNGReader(self.annotations_path / "test")
            test_dataset = DatasetForSAM(
                readers=[test_img_reader, test_label_reader],
                transforms=self.transforms
            )
            self.datasets["test"] = test_dataset
            self.datasets["predict"] = test_dataset

        else:
            raise ValueError(f"Invalid stage: {stage}")
    
    def custom_collate_fn(self, batch):
        """
        Custom collate function for DataLoader to return a list of dictionaries.
        """
        return batch 

    def train_dataloader(self):
        return DataLoader(
            self.datasets["train"],
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True,
            collate_fn=self.custom_collate_fn
        )

    def val_dataloader(self):
        return DataLoader(
            self.datasets["val"],
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            collate_fn=self.custom_collate_fn
        )

    def test_dataloader(self):
        return DataLoader(
            self.datasets["test"],
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            collate_fn=self.custom_collate_fn
        )

    def predict_dataloader(self):
        return DataLoader(
            self.datasets["predict"],
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            collate_fn=self.custom_collate_fn
        )

## init

In [8]:
data_module = DataModule(
    train_path=train_path,
    annotations_path=annotation_path,
    transforms=Padding(height, width),
    batch_size=batch_size
)

num_workers:  16


In [9]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
print("Using device: ", device)

# experiment 1
model = Sam(
    vit_type=vit_model,
    checkpoint=checkpoint_path,
    num_multimask_outputs=num_classes, # default: 3
    iou_head_depth=num_classes, # default: 3
    # apply_freeze=apply_freeze,
    # apply_adapter=apply_adapter
).to(device)

# experiment 3
# model = Sam(
#     vit_type=vit_model,
#     checkpoint=checkpoint_path,
#     num_multimask_outputs=num_classes, # default: 3
#     iou_head_depth=num_classes, # default: 3
#     apply_freeze={"image_encoder": False, "prompt_encoder": False, "mask_decoder": False},
#     # apply_adapter=apply_adapter,
#     train_metrics={"mIoU": JaccardIndex(task="multiclass", num_classes=num_classes)},
#     val_metrics={"mIoU": JaccardIndex(task="multiclass", num_classes=num_classes)},
#     test_metrics={"mIoU": JaccardIndex(task="multiclass", num_classes=num_classes)},
#     # multimask_output=multimask_output,
#     # loss_fn=DiceCELoss() # if multimask_output is false
# ).to(device)

model.eval()

Using device:  cuda


/home/vscode/.local/lib/python3.10/site-packages/minerva/models/nets/image/sam.py:2343: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Prompt Encoder freeze!


Sam(
  (loss_fn): CrossEntropyLoss()
  (model): _SAM(
    (image_encoder): ImageEncoderViT(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (blocks): ModuleList(
        (0-11): 12 x Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (lin1): Linear(in_features=768, out_features=3072, bias=True)
            (lin2): Linear(in_features=3072, out_features=768, bias=True)
            (act): GELU(approximate='none')
          )
        )
      )
      (neck): Sequential(
        (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): LayerNorm2d()
        (2): Conv2d(256, 256,

## AUC class

In [10]:
class AUC_calculate_v1():
    def __init__(self,
                 model,
                 dataloader,
                 num_points:int=3 ,
                 experiment_num:int=0,
                 ):
        self.model = model
        self.model.eval() # seta para inferencia
        self.dataloader = dataloader
        self.num_points = num_points
        self.experiment_num = experiment_num
        self.set_points() # reseta pontos

        self.miou_metric = JaccardIndex(task="multiclass", num_classes=2).to(self.model.device) # binario
    
    def set_points(self):
        # Inicialize os acumuladores como arrays vazios
        self.accumulated_coords = np.empty((0, 2), dtype=int)  # Nx2 array
        self.accumulated_labels = np.empty((0,), dtype=int)   # Array de comprimento N
    
    def process(self):
        self.results = pd.DataFrame(columns=['sample_id', 'facie_id', 'accumulated_point', 'iou', 'num_points'])

        with torch.no_grad():
            # for batch_idx, batch in enumerate(self.dataloader):
            for batch_idx, batch in enumerate(tqdm(self.dataloader, desc="Processando batches")):
                for item in batch:
                    img = item['image']
                    # print(f"Intervalo original: min={img.min()}, max={img.max()}")
                    if img.shape[0] == 1:
                        img = img.repeat(3, 1, 1)
                    img = (img * 255).clamp(0, 255).to(torch.uint8)
                    # print(f"Intervalo após multiplicação por 255: min={img.min()}, max={img.max()}")
                    # _, png_img = cv2.imencode('.png', img)
                    # decoded_image = cv2.imdecode(np.frombuffer(png_img, np.uint8), cv2.IMREAD_UNCHANGED)
                    # decoded_image = torch.from_numpy(decoded_image).permute(2, 0, 1).float()
                    image = img.to(self.model.device)
                    label = item['label'].squeeze(0).numpy()#.to(self.model.device)
                #     plt.figure(figsize=(6, 6))  # Define o tamanho da figura
                #     plt.imshow(label, cmap='gray')  # Mostra a imagem, com escala de cinza (ou RGB se for colorida)
                #     plt.title("label")  # Adiciona o título
                #     plt.axis('off')  # Desliga os eixos para uma visualização limpa
                #     plt.show()
                #     break
                # break

                    num_facies = np.unique(label) # num de facies da amostra
                    point_type = 'positive' # inicia com ponto positivo, depois pode mudar para 'negative'

                    for i, facie in enumerate(num_facies):
                        region = np.zeros_like(label, dtype=np.uint8) # [H,W]
                        region[label == facie] = 1
                        real_label = region
                        # plt.figure(figsize=(6, 6))  # Define o tamanho da figura
                        # plt.imshow(region, cmap='gray')  # Mostra a imagem, com escala de cinza (ou RGB se for colorida)
                        # plt.title("region")  # Adiciona o título
                        # plt.axis('off')  # Desliga os eixos para uma visualização limpa
                        # plt.show()
                        # break

                        for point in range(self.num_points):
                            # calculando centro da regiao e retornando sua coordenada
                            point_coords, point_labels = self.calculate_center_region(region=region, point_type=point_type)
                        
                            # definindo amostra (batch) a ser inserido no modelo
                            batch = {
                                'image': image,
                                'label': label,
                                'original_size': (int(image.shape[1]), int(image.shape[2])),
                                'point_coords': torch.tensor(point_coords, dtype=torch.long).unsqueeze(0).to(self.model.device),
                                'point_labels': torch.tensor(point_labels, dtype=torch.long).unsqueeze(0).to(self.model.device)
                            }
                        
                            # Inferência
                            outputs = self.model([batch], multimask_output=multimask_output) # batch tem que ser uma lista de dict. multimask_output é dado no init do model
                            
                            # calcular IoU
                            gt_tensor = torch.tensor(real_label).to(self.model.device)  # Converta para tensor 2D e mova para GPU
                            pred_tensor = torch.tensor(outputs[0]['masks'].squeeze()).to(self.model.device)  # Remover a dimensão extra e mover para GPU
                            
                            iou_score = self.miou_metric(pred_tensor, gt_tensor)

                            # print("real_label shape: ", real_label.shape)
                            # print('pred shape: ', outputs[0]['masks'].squeeze().numpy().shape)
                            diff, new_point_type = self.calculate_diff_label_pred(label=real_label, pred=outputs[0]['masks'].squeeze().cpu().numpy())
                        
                            # salvando progresso
                            new_row = pd.DataFrame([{
                                'sample_id': batch_idx,
                                'facie_id': facie,
                                'accumulated_point': point + 1,
                                'iou': iou_score.item(),
                                'num_points': self.num_points
                            }])

                            self.results = pd.concat([self.results, new_row], ignore_index=True)
                        
                            # plot experimental a cada 50 amostras
                            # if idx % 50 == 0:
                            # plot_all(
                            #     image=image.permute(1, 2, 0).cpu(),
                            #     label=real_label,
                            #     pred=outputs[0]['masks'].squeeze().cpu().numpy(),
                            #     diff=diff,
                            #     score=iou_score,
                            #     point_coords=self.accumulated_coords,
                            #     point_labels=self.accumulated_labels
                            # )
                            region = diff # [H,W], atualiza para a proxima regiao
                            point_type = new_point_type # 'positive' ou 'negative', atualiza para a proxima regiao
                            # break # testa só 1 ponto
                        point_type = 'positive' # reinicia tipo do primeiro ponto (sempre deve ser positivo o primeiro)
                        self.set_points() # reinicia empilhamento de pontos
                        # break # testa só 1 facie
                    # break # testar só uma amostra
                # break # testar só uma amostra
            
            os.makedirs('save_test', exist_ok=True)
            self.results.to_csv(f'save_test/iou_results_{self.experiment_num}.csv', index=False)

    def calculate_center_region(self, region: np.array, point_type: str, min_distance: int = 10):
        """
        Calcula o centroide da maior região de pixels brancos de uma imagem binária,
        deslocando horizontalmente o ponto se ele estiver próximo demais dos acumulados.

        Args:
            region (np.array): Imagem binária com a região de interesse (pixels brancos).
            point_type (str): Tipo do ponto ('positive' ou 'negative').
            min_distance (int): Distância mínima permitida entre pontos.

        Returns:
            point_coords (np.ndarray): Array Nx2 de pontos acumulados.
            point_labels (np.ndarray): Array N de rótulos acumulados.
        """
        if not isinstance(region, np.ndarray):
            raise TypeError("region needs to be a NumPy array.")
        
        # Encontrar as componentes conectadas
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(region, connectivity=8)

        if num_labels < 2:  # Apenas fundo e nenhuma região branca
            raise ValueError("No connected white regions found in the binary image.")
        
        # Ignorar o rótulo 0 (fundo), pegar a maior componente conectada
        largest_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
        center_x, center_y = centroids[largest_label]
        center_x, center_y = int(center_x), int(center_y)
        new_coords = np.array([[center_x, center_y]])

        # Verificar se o ponto está muito próximo dos anteriores
        if self.accumulated_coords.shape[0] > 0:
            distances = np.sqrt(np.sum((self.accumulated_coords - new_coords) ** 2, axis=1))
            if np.any(distances < min_distance):
                # print("Ponto muito próximo ao anterior, deslocando horizontalmente...")
                
                # Tentar deslocar o ponto horizontalmente dentro da região branca
                region_height, region_width = region.shape
                # Tenta deslocar horizontalmente usando o min_distance
                for delta_x in range(min_distance, region_width, min_distance):  # Incrementa em min_distance
                    candidate_x_right = center_x + delta_x
                    candidate_x_left = center_x - delta_x

                    # Verifica primeiro para direita, depois para esquerda
                    if candidate_x_right < region_width and region[center_y, candidate_x_right] > 0:
                        center_x = candidate_x_right
                        break
                    elif candidate_x_left >= 0 and region[center_y, candidate_x_left] > 0:
                        center_x = candidate_x_left
                        break

                new_coords = np.array([[center_x, center_y]])

        # Definir o rótulo (positivo ou negativo)
        if point_type == 'positive':
            new_labels = np.array([1])
        elif point_type == 'negative':
            new_labels = np.array([0])
        else:
            raise ValueError("Invalid point_type. Must be 'positive' or 'negative'.")

        # Acumular os resultados
        self.accumulated_coords = np.vstack([self.accumulated_coords, new_coords])
        self.accumulated_labels = np.hstack([self.accumulated_labels, new_labels])

        return self.accumulated_coords, self.accumulated_labels
    
    def calculate_diff_label_pred(self, label:np.array, pred:np.array):
        """
        Calcula a diferença entre duas imagens binárias e determina se a área externa ou interna é maior.

        Args:
            label (np.array): Imagem binária de referência (label).
            pred (np.array): Imagem binária predita (pred).

        Returns:
            diff_colored (np.array): Imagem colorida representando as diferenças.
            point_type (str): 'negative' se a área externa for maior, 'positive' se a interna for maior.
        """
        if label.shape != pred.shape:
            raise ValueError("Label and Pred images have differents shapes. Check it before call calculate_dif_label_pred() function.")

        # Máscaras para regiões de diferença
        mask_outward = (label > pred)  # Diferença para fora -> Vermelho
        mask_inward = (label < pred)  # Diferença para dentro -> Azul

        area_outward = np.sum(mask_outward)
        area_inward = np.sum(mask_inward)

        diff_binary = teste1 = teste2 = np.zeros(label.shape, dtype=np.uint8) # [H,W]

        # fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        # teste1[mask_outward] = 1
        # axes[0].imshow(teste1)
        # axes[0].set_title('Image 1')
        # axes[0].axis('off')
        # teste2[mask_inward] = 1
        # axes[1].imshow(teste2)
        # axes[1].set_title('Image 2')
        # axes[1].axis('off')
        # plt.tight_layout()
        # plt.show()

        # Comparar as áreas
        if area_outward > area_inward:
            diff_binary[mask_outward] = 1
            point_type = 'positive'
        else:
            diff_binary[mask_inward] = 1
            point_type = 'negative'
        
        return diff_binary, point_type

## exec test

In [11]:
data_module.setup(stage='predict')  # Configura os dados para inferência
miou_metric = JaccardIndex(task="multiclass", num_classes=num_classes) # Inicializando a métrica de mIoU
predict_dataloader = data_module.predict_dataloader()

In [12]:
algorithm = AUC_calculate_v1(
    model=model,
    dataloader=predict_dataloader,
    num_points=10,
)

algorithm.process()

Processando batches:   0%|          | 0/400 [00:00<?, ?it/s]/tmp/ipykernel_213334/408153985.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_tensor = torch.tensor(outputs[0]['masks'].squeeze()).to(self.model.device)  # Remover a dimensão extra e mover para GPU
/tmp/ipykernel_213334/408153985.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, new_row], ignore_index=True)
Processando batches: 100%|██████████| 400/400 [21:10<00:00,  3.18s/it]
